In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [7]:
df = pd.read_csv("filtered_dataset.csv")

In [8]:
# === Drop duplicate sequences ===
df = df.drop_duplicates(subset="sequence")

# Drop rows where any column has a missing value
df = df.dropna()

# === Save the final filtered dataset ===
df.to_csv("filtered_dataset.csv", index=False)

print(f"Filtered dataset saved as filtered_dataset.csv with {len(df)} rows.")

Filtered dataset saved as filtered_dataset.csv with 154 rows.
